# Asymptomatic transmission

In [1]:
import numpy
import networkx
import epyc
import epydemic
import pandas
import mpmath

import matplotlib
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import matplotlib.pyplot as plt
import seaborn
matplotlib.style.use('seaborn')

In [3]:
lab = epyc.ClusterLab(profile='sif', notebook=epyc.JSONLabNotebook('datasets/seir-er.json'))
with lab.sync_imports():
    import time
    import networkx
    import epyc
    import epydemic
    import numpy
print('{n} engines available'.format(n = lab.numberOfEngines()))

importing time on engine(s)
importing networkx on engine(s)
importing epyc on engine(s)
importing epydemic on engine(s)
importing numpy on engine(s)
6 engines available


In [3]:
# from https://nbviewer.jupyter.org/gist/minrk/4470122
def pxlocal(line, cell):
    ip = get_ipython()
    ip.run_cell_magic("px", line, cell)
    ip.run_cell(cell)
get_ipython().register_magic_function(pxlocal, "cell")    

In [4]:
%%pxlocal

class ERNetworkDynamics(epydemic.StochasticDynamics):
    
    # Experimental parameters
    N = 'N'
    KMEAN = 'kmean'
    
    def __init__(self, p):
        super(ERNetworkDynamics, self).__init__(p)
        
    def configure(self, params):
        super(ERNetworkDynamics, self).configure(params)
        
        # build a random ER network with the given parameters
        N = params[self.N]
        kmean = params[self.KMEAN]
        pEdge = (kmean + 0.09) / N
        g = networkx.gnp_random_graph(N, pEdge)
        self.setNetworkPrototype(g)

In [2]:
class MonitoredSIR(epydemic.SIR, epydemic.Monitor):
 
    def __init__(self):
        super(MonitoredSIR, self).__init__()
        
    def build(self, params):
        '''Build the observation process.
        
        :param params: the experimental parameters'''
        super(MonitoredSIR, self).build(params)

        # also monitor other compartments
        self.trackNodesInCompartment(epydemic.SIR.SUSCEPTIBLE)
        self.trackNodesInCompartment(epydemic.SIR.REMOVED)

In [3]:
class MonitoredSEIR(epydemic.SEIR, epydemic.Monitor):
 
    def __init__(self):
        super(MonitoredSEIR, self).__init__()
        
    def build(self, params):
        '''Build the observation process.
        
        :param params: the experimental parameters'''
        super(MonitoredSEIR, self).build(params)

        # also monitor other compartments
        self.trackNodesInCompartment(epydemic.SIR.SUSCEPTIBLE)
        self.trackNodesInCompartment(epydemic.SIR.REMOVED)

In [5]:
# test network
lab[ERNetworkDynamics.N] = 10000
lab[ERNetworkDynamics.KMEAN] = 40

# disease parameters
lab[epydemic.SEIR.P_EXPOSED] = 0.001
lab[epydemic.SEIR.P_REMOVE] = 0.002
lab[epydemic.SEIR.P_INFECT_ASYMPTOMATIC] = numpy.linspace(0.00001, 0.0002, num=50)
lab[epydemic.SEIR.P_INFECT_SYMPTOMATIC] = 0.000075   # middle of the phase transition
lab[epydemic.SEIR.P_SYMPTOMS] = 0.002

In [6]:
m = epydemic.SEIR()
e = ERNetworkDynamics(m)
rc = lab.runExperiment(epyc.RepeatedExperiment(epyc.RepeatedExperiment(e, 10), 10))

In [7]:
lab.readyFraction()

RemoteError: ModuleNotFoundError(No module named 'epydemic.seir_model')